## Plan
1. use 2d unet from [segmentation_models.pytorch](https://github.com/qubvel/segmentation_models.pytorch) repo
2. add cv with n folds (let's start with n = 5)
3. train on:
  - resized image
  - image fragments


In [1]:
import torch
import numpy as np
import pandas as pd
import os
import random
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm.notebook import tqdm
import cv2

In [2]:
#!pip install --upgrade segmentation-models-pytorch
SEED = 2768
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

In [3]:
import segmentation_models_pytorch as smp


In [4]:
with np.load('../data/data_train.npz') as dataset:
        train_dataset = dataset['data']
        
with np.load('../data/data_test_1.npz') as dataset:
        test_dataset = dataset['data']

with np.load('../data/labels_train.npz') as labels:
        train_labels = labels['labels']

In [5]:
def fast_glcm(img, vmin=0, vmax=255, nbit=8, kernel_size=5):
    mi, ma = vmin, vmax
    ks = kernel_size
    h,w = img.shape

    # digitize
    bins = np.linspace(mi, ma+1, nbit+1)
    gl1 = np.digitize(img, bins) - 1
    gl2 = np.append(gl1[:,1:], gl1[:,-1:], axis=1)

    # make glcm
    glcm = np.zeros((nbit, nbit, h, w), dtype=np.uint8)
    for i in range(nbit):
        for j in range(nbit):
            mask = ((gl1==i) & (gl2==j))
            glcm[i,j, mask] = 1

    kernel = np.ones((ks, ks), dtype=np.uint8)
    for i in range(nbit):
        for j in range(nbit):
            glcm[i,j] = cv2.filter2D(glcm[i,j], -1, kernel)

    glcm = glcm.astype(np.float32)
    return glcm

def fast_glcm_max(img, vmin=0, vmax=255, nbit=2, ks=5):
    '''
    calc glcm max
    '''
    glcm = fast_glcm(img, vmin, vmax, nbit, ks)
    max_  = np.max(glcm, axis=(0,1))
    return max_

# Preprocessing our image, inclusing  applying fast_glcm_max function, resizing and normalizing pixel values 
def process_img_label(img, label=None):

    img = fast_glcm_max(img)

    img = np.expand_dims(img, axis=2)#.astype('float32')


    img = cv2.resize(img, (224, 224))


    img = cv2.normalize(img, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX).astype(np.uint8)

    try:
        label = np.expand_dims(label, axis=2).astype(np.uint8)
        label = cv2.resize(label, (224, 224))

        return img, label

    except:
        return img

In [6]:
test_dataset.shape

(1006, 782, 251)

In [7]:
training_img_data = []
test_img_data = []
training_label_data = []

for i in tqdm(range(0, 590)):
    img = train_dataset[:, :, i]
    label = train_labels[:, :, i]

    img, label = process_img_label(img, label)

    training_img_data.append(img) 
    training_label_data.append(label)
for i in tqdm(range(0, 251)):
    img = test_dataset[:, :, i]
    img = process_img_label(img)
    test_img_data.append(img)

/home/lacemaker/anaconda3/envs/data_env/lib/python3.7/site-packages/ipykernel_launcher.py:48: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.


In [8]:
training_img_data = np.asarray(training_img_data)
test_img_data = np.asarray(test_img_data)
training_label_data = np.asarray(training_label_data)
training_img_data = np.expand_dims(training_img_data, 1)
test_img_data = np.expand_dims(test_img_data, 1)
training_img_data.shape, training_label_data.shape, test_img_data.shape

((590, 1, 224, 224), (590, 224, 224), (251, 1, 224, 224))

In [9]:
import solt
import solt.transforms as slt

stream = solt.Stream([
    slt.Rotate(angle_range=(0, 360), p=1, padding='r'),
     slt.Shear(range_x=0.3, range_y=0.8, p=0.5, padding='r'),
     slt.Flip(axis=1, p=0.5),
    slt.Flip(axis=0, p=0.5),
    #  slt.IntensityRemap(),
    slt.Brightness(brightness_range=(0, 1)),
     slt.Pad(200),
     slt.Scale(range_x=(0.8, 1.3), padding='r', range_y=(0.8, 1.3), same=False, p=0.5),
     slt.Contrast(p=0.5),
    #   solt.SelectiveStream([
    #     slt.CutOut(40, p=1),
    #     slt.CutOut(50, p=1),
    #     slt.CutOut(10, p=1),
    #     solt.Stream(),
    #     solt.Stream(),
    # ], n=3),
    # slt.Crop((224, 224)),
    solt.SelectiveStream([
        slt.GammaCorrection(gamma_range=0.5, p=1),
        slt.Noise(gain_range=0.1, p=1),
        slt.Blur()    
    ], n=3)
])

In [10]:
ENCODER = 'resnet34' #'se_resnext50_32x4d'
ENCODER_WEIGHTS = 'imagenet'

ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multicalss segmentation
DEVICE = 'cuda'

# create segmentation model with pretrained encoder

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)



In [11]:
#model(torch.randn(10, 1, 224, 224)).shape

In [12]:
training_img_data.shape

(590, 1, 224, 224)

In [13]:
from sklearn.model_selection import KFold

In [14]:
import albumentations as albu

def get_training_augmentation(size=224):
    train_transform = [

        albu.HorizontalFlip(p=0.5),

        albu.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0),

        albu.PadIfNeeded(min_height=size, min_width=size, always_apply=True, border_mode=0),
        albu.RandomCrop(height=size, width=size, always_apply=True),

        albu.IAAAdditiveGaussianNoise(p=0.2),
        albu.IAAPerspective(p=0.5),

        albu.OneOf(
            [
                albu.CLAHE(p=1),
                albu.RandomBrightness(p=1),
                albu.RandomGamma(p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.IAASharpen(p=1),
                albu.Blur(blur_limit=3, p=1),
                albu.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.RandomContrast(p=1),
                albu.HueSaturationValue(p=1),
            ],
            p=0.9,
        ),
    ]
    return albu.Compose(train_transform)


def get_validation_augmentation(do_pad=False):
    """Add paddings to make image shape divisible by 32"""
    if do_pad:
        test_transform = [
        albu.PadIfNeeded(384, 480)
        ]
    else:
        test_transform = []
    return albu.Compose(test_transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

In [15]:
from torch.utils.data import Dataset, DataLoader

#img_aug, mask_aug = stream({'image': train_data[:, 0], 'mask':train_labels[:,0]}, return_torch=False, ).data
class SeismicDataset(Dataset):
    def __init__(self, data, mask_data=None, transform=None, augmentation=None, preprocessing=None, labels=[]):
        self.data = data
        self.mask_data = mask_data
        self.transform = transform
        self.augmentation=augmentation
        self.preprocessing = preprocessing
        self.labels=labels
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        img = self.data[idx, 0]
        img_new = np.zeros(img.shape + (3,), dtype=img.dtype)
        for i in range(3):
            img_new[:, :, i] = img[:, :]
        img = img_new
        #print(img.shape, img.dtype)
        if self.mask_data is None:
            if self.transform is not None:
                img = self.transform(img)
            if self.augmentation:
                sample = self.augmentation(image=img)
                img = sample['image']

            # apply preprocessing
            if self.preprocessing:
                sample = self.preprocessing(image=img)
                img = sample['image']
            return img
        mask = self.mask_data[idx]
        masks = np.zeros(mask.shape+(len(self.labels), ), dtype=np.float)
        for i, label in enumerate(self.labels):
            masks[:, :, i] = mask[:, :] == label
            #print(np.sum(mask==label))
        #if self.transform is not None:
        #    img, label = self.transform(img, label)
        if self.augmentation:
            sample = self.augmentation(image=img, mask=masks)
            img, masks = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=img, mask=masks)
            img, masks = sample['image'], sample['mask']
            
        return img, masks

In [16]:
# train model for 40 epochs
def do_train(model, train_epoch, valid_epoch, train_loader, val_loader, n_epochs=40, fold=0):
    max_score = 0

    for i in range(0, n_epochs):

        print('\nEpoch: {}'.format(i))
        train_logs = train_epoch.run(train_loader)
        valid_logs = valid_epoch.run(val_loader)

        # do something (save model, change lr, etc.)
        if max_score < valid_logs['iou_score']:
            max_score = valid_logs['iou_score']
            torch.save(model, f'./solution1/best_model_{fold}.pth')
            print('Model saved!')

        if i == 25:
            optimizer.param_groups[0]['lr'] = 1e-5
            print('Decrease decoder learning rate to 1e-5!')

In [17]:
loss = smp.utils.losses.DiceLoss()
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
]

ACTIVATION = 'softmax2d' # could be None for logits or 'softmax2d' for multicalss segmentation
NFOLDS=5
labels = np.unique(training_label_data)

In [18]:

kf = KFold(random_state=SEED, n_splits=NFOLDS, shuffle=True)
for fold, (train_ids, val_ids) in enumerate(kf.split(training_img_data)):
    train_data = training_img_data[train_ids]
    train_labels = training_label_data[train_ids]
    val_data = training_img_data[val_ids]
    val_labels = training_label_data[val_ids]
    
    train_ds = SeismicDataset(
        train_data, train_labels,
        preprocessing=get_preprocessing(preprocessing_fn),
        augmentation=get_training_augmentation(),
        labels=labels
    )
    val_ds = SeismicDataset(
        val_data, val_labels,
        preprocessing=get_preprocessing(preprocessing_fn),
        augmentation=get_validation_augmentation(),
        labels=labels
    )
    
    train_loader = DataLoader(train_ds, batch_size=32)
    val_loader = DataLoader(val_ds, batch_size=4)
    model = smp.Unet(
        ENCODER, classes=len(labels), in_channels=3,
        encoder_weights=ENCODER_WEIGHTS,
        activation=ACTIVATION,
    )
    
    optimizer = torch.optim.Adam([ 
        dict(params=model.parameters(), lr=0.0001),
    ])

    train_epoch = smp.utils.train.TrainEpoch(
        model, 
        loss=loss, 
        metrics=metrics, 
        optimizer=optimizer,
        device=DEVICE,
        verbose=True,
    )

    valid_epoch = smp.utils.train.ValidEpoch(
        model, 
        loss=loss, 
        metrics=metrics, 
        device=DEVICE,
        verbose=True,
    )

    do_train(model, train_epoch, valid_epoch, train_loader, val_loader, fold=fold)
    #break


Epoch: 0
valid: 100%|██████████| 30/30 [00:00<00:00, 51.47it/s, dice_loss - 0.7736, iou_score - 0.08398]
Model saved!

Epoch: 1
valid: 100%|██████████| 30/30 [00:00<00:00, 51.06it/s, dice_loss - 0.7013, iou_score - 0.1951]
Model saved!

Epoch: 2
valid: 100%|██████████| 30/30 [00:00<00:00, 50.10it/s, dice_loss - 0.6391, iou_score - 0.2614]
Model saved!

Epoch: 3
valid: 100%|██████████| 30/30 [00:00<00:00, 50.87it/s, dice_loss - 0.5893, iou_score - 0.3124]
Model saved!

Epoch: 4
valid: 100%|██████████| 30/30 [00:00<00:00, 49.67it/s, dice_loss - 0.5555, iou_score - 0.3319]
Model saved!

Epoch: 5
valid: 100%|██████████| 30/30 [00:00<00:00, 49.54it/s, dice_loss - 0.5103, iou_score - 0.3683]
Model saved!

Epoch: 6
valid: 100%|██████████| 30/30 [00:00<00:00, 48.17it/s, dice_loss - 0.4689, iou_score - 0.3689]
Model saved!

Epoch: 7
valid: 100%|██████████| 30/30 [00:00<00:00, 49.32it/s, dice_loss - 0.4421, iou_score - 0.3694]
Model saved!

Epoch: 8
valid: 100%|██████████| 30/30 [00:00<00:00, 4

valid: 100%|██████████| 30/30 [00:00<00:00, 50.99it/s, dice_loss - 0.1193, iou_score - 0.852] 

Epoch: 39
valid: 100%|██████████| 30/30 [00:00<00:00, 51.47it/s, dice_loss - 0.1181, iou_score - 0.8539]
Model saved!

Epoch: 0
valid: 100%|██████████| 30/30 [00:00<00:00, 52.20it/s, dice_loss - 0.8323, iou_score - 0.02898]
Model saved!

Epoch: 1
valid: 100%|██████████| 30/30 [00:00<00:00, 49.07it/s, dice_loss - 0.7736, iou_score - 0.02664]

Epoch: 2
valid: 100%|██████████| 30/30 [00:00<00:00, 49.89it/s, dice_loss - 0.6579, iou_score - 0.2524]
Model saved!

Epoch: 3
valid: 100%|██████████| 30/30 [00:00<00:00, 50.90it/s, dice_loss - 0.5493, iou_score - 0.3926]
Model saved!

Epoch: 4
valid: 100%|██████████| 30/30 [00:00<00:00, 50.88it/s, dice_loss - 0.5347, iou_score - 0.4329]
Model saved!

Epoch: 5
valid: 100%|██████████| 30/30 [00:00<00:00, 50.18it/s, dice_loss - 0.4487, iou_score - 0.5008]
Model saved!

Epoch: 6
valid: 100%|██████████| 30/30 [00:00<00:00, 51.46it/s, dice_loss - 0.4095, iou_

valid: 100%|██████████| 30/30 [00:00<00:00, 50.39it/s, dice_loss - 0.1309, iou_score - 0.8365]

Epoch: 38
valid: 100%|██████████| 30/30 [00:00<00:00, 50.63it/s, dice_loss - 0.1307, iou_score - 0.8386]
Model saved!

Epoch: 39
valid: 100%|██████████| 30/30 [00:00<00:00, 50.82it/s, dice_loss - 0.1298, iou_score - 0.8411]
Model saved!

Epoch: 0
valid: 100%|██████████| 30/30 [00:00<00:00, 50.47it/s, dice_loss - 0.6915, iou_score - 0.1968]
Model saved!

Epoch: 1
valid: 100%|██████████| 30/30 [00:00<00:00, 48.17it/s, dice_loss - 0.5362, iou_score - 0.4151]
Model saved!

Epoch: 2
valid: 100%|██████████| 30/30 [00:00<00:00, 48.75it/s, dice_loss - 0.4459, iou_score - 0.5185]
Model saved!

Epoch: 3
valid: 100%|██████████| 30/30 [00:00<00:00, 49.10it/s, dice_loss - 0.3599, iou_score - 0.589] 
Model saved!

Epoch: 4
valid: 100%|██████████| 30/30 [00:00<00:00, 49.08it/s, dice_loss - 0.3524, iou_score - 0.5727]

Epoch: 5
valid: 100%|██████████| 30/30 [00:00<00:00, 49.95it/s, dice_loss - 0.3349, iou_s

valid: 100%|██████████| 30/30 [00:00<00:00, 49.29it/s, dice_loss - 0.1162, iou_score - 0.8273]

Epoch: 37
valid: 100%|██████████| 30/30 [00:00<00:00, 50.80it/s, dice_loss - 0.1174, iou_score - 0.8288]

Epoch: 38
valid: 100%|██████████| 30/30 [00:00<00:00, 50.44it/s, dice_loss - 0.1156, iou_score - 0.8304]
Model saved!

Epoch: 39
valid: 100%|██████████| 30/30 [00:00<00:00, 51.07it/s, dice_loss - 0.1154, iou_score - 0.8313]
Model saved!

Epoch: 0
valid: 100%|██████████| 30/30 [00:00<00:00, 50.65it/s, dice_loss - 0.749, iou_score - 0.09448] 
Model saved!

Epoch: 1
valid: 100%|██████████| 30/30 [00:00<00:00, 49.68it/s, dice_loss - 0.6112, iou_score - 0.3069]
Model saved!

Epoch: 2
valid: 100%|██████████| 30/30 [00:00<00:00, 48.64it/s, dice_loss - 0.5545, iou_score - 0.3556]
Model saved!

Epoch: 3
valid: 100%|██████████| 30/30 [00:00<00:00, 50.36it/s, dice_loss - 0.5309, iou_score - 0.3702]
Model saved!

Epoch: 4
valid: 100%|██████████| 30/30 [00:00<00:00, 49.94it/s, dice_loss - 0.4925, iou

valid: 100%|██████████| 30/30 [00:00<00:00, 48.00it/s, dice_loss - 0.1155, iou_score - 0.8382]

Epoch: 35
valid: 100%|██████████| 30/30 [00:00<00:00, 48.32it/s, dice_loss - 0.1151, iou_score - 0.8403]
Model saved!

Epoch: 36
valid: 100%|██████████| 30/30 [00:00<00:00, 46.66it/s, dice_loss - 0.1143, iou_score - 0.8396]

Epoch: 37
valid: 100%|██████████| 30/30 [00:00<00:00, 48.70it/s, dice_loss - 0.1134, iou_score - 0.8425]
Model saved!

Epoch: 38
valid: 100%|██████████| 30/30 [00:00<00:00, 51.68it/s, dice_loss - 0.1136, iou_score - 0.843] 
Model saved!

Epoch: 39
valid: 100%|██████████| 30/30 [00:00<00:00, 49.84it/s, dice_loss - 0.1139, iou_score - 0.8444]
Model saved!

Epoch: 0
valid: 100%|██████████| 30/30 [00:00<00:00, 48.16it/s, dice_loss - 0.7687, iou_score - 0.1077]
Model saved!

Epoch: 1
valid: 100%|██████████| 30/30 [00:00<00:00, 47.54it/s, dice_loss - 0.665, iou_score - 0.1807] 
Model saved!

Epoch: 2
valid: 100%|██████████| 30/30 [00:00<00:00, 45.79it/s, dice_loss - 0.5935, io

valid: 100%|██████████| 30/30 [00:00<00:00, 49.82it/s, dice_loss - 0.1001, iou_score - 0.858] 
Model saved!

Epoch: 34
valid: 100%|██████████| 30/30 [00:00<00:00, 50.89it/s, dice_loss - 0.09839, iou_score - 0.8602]
Model saved!

Epoch: 35
valid: 100%|██████████| 30/30 [00:00<00:00, 48.06it/s, dice_loss - 0.09967, iou_score - 0.8587]

Epoch: 36
valid: 100%|██████████| 30/30 [00:00<00:00, 50.42it/s, dice_loss - 0.09847, iou_score - 0.8613]
Model saved!

Epoch: 37
valid: 100%|██████████| 30/30 [00:00<00:00, 50.79it/s, dice_loss - 0.09635, iou_score - 0.8616]
Model saved!

Epoch: 38
valid: 100%|██████████| 30/30 [00:00<00:00, 48.22it/s, dice_loss - 0.0964, iou_score - 0.8621] 
Model saved!

Epoch: 39
valid: 100%|██████████| 30/30 [00:00<00:00, 50.09it/s, dice_loss - 0.09625, iou_score - 0.8625]
Model saved!


In [19]:
#out = model(torch.from_numpy(train_data).float())

In [20]:
training_img_data.shape

(590, 1, 224, 224)

In [21]:
test_img_data.shape

(251, 1, 224, 224)

In [22]:
test_ds = SeismicDataset(
        test_img_data,
        preprocessing=get_preprocessing(preprocessing_fn),
        augmentation=get_validation_augmentation(),
        labels=labels
)
val_loader = DataLoader(test_ds, batch_size=4)

In [24]:
with torch.no_grad():
    all_predictions = []
    for fold in np.arange(NFOLDS):
        fold_predictions = []
        model = torch.load(f'./solution1/best_model_{fold}.pth')
        model = model.to(DEVICE)
        for batch in val_loader:
            predictions = model(batch.to(DEVICE))
            predictions = predictions.detach().cpu().numpy()
            fold_predictions.append(predictions)
        fold_predictions = np.concatenate(fold_predictions, 0)
        all_predictions.append(fold_predictions)
        del model
    #torch.load()
    #for batch in val_loader:

In [25]:
predictions = np.mean(all_predictions, 0)

In [26]:
all_predictions = []
for pred in predictions:
    p = cv2.resize(np.moveaxis(pred, 0, -1), (782, 1006))
    p = np.argmax(p, -1)
    all_predictions.append(labels[p])
all_predictions = np.stack(all_predictions, -1)

In [27]:
with np.load('../data/sample_submission_1.npz') as dataset:
    print(list(dataset.keys()))
    sample = dataset['prediction']

['prediction']


In [28]:
assert sample.shape==all_predictions.shape

In [29]:
np.savez_compressed("solution1/submission_solution1.npz", prediction=all_predictions)